**The cell below imports all the necessary packages required for this classification task**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout,Dense, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Lambda
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, roc_curve, auc
import itertools
from sklearn import metrics
from keras.models import load_model
import os
import shutil
from keras import backend as K
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

**The below cell initialized the width, height of input images along with the directories for training and testing data, the batch size and number of epochs(iterations)**

In [ ]:
img_width, img_height = 64, 64

train_data_dir = '../input/modiified/Modified_Dataset/Train'
test_data_dir = '../input/modiified/Modified_Dataset/Test'
nb_train_samples = 1050
nb_validation_samples = 144
epochs = 50
batch_size = 32

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

**This cell generates more training data by performing data augmentation, also performs data normalization by dividing them by 255**

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.25,
    height_shift_range=0.25,
    horizontal_flip=True,
    vertical_flip=True)
test_datagen = ImageDataGenerator(rescale=1. / 255)


**Here the training and testing (here testing and validation are used as same) data batches are prepared by giving the directory to the data and other necessary parameters**

In [ ]:
train_batches = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    classes=['BENIGN', 'MALIGNANT', 'NORMAL'])
    #color_mode="grayscale")

**Same thing is performed as the training batch before**

In [ ]:
test_batches = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    classes=['BENIGN', 'MALIGNANT', 'NORMAL'])
    #color_mode="grayscale")

**Here the Convolutional Neural Network model is built. You can use different number of layers,tune different value like dropout value from 0.5 to 0.6 or 0.25 or whatever value that may give the best result.**

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape, padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.5))

**Just copy and paste the below cell block of three lines for an additional number of layer. You can change the value 32 to 64 or 128 or whatever value which may give the best output.**

In [ ]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

**The Conv2D layers above are responsible for extracting the features and the layer below i.e the dense layers are responsible to perform the classification. Simillarly, you can change the number of nodes in the dense layer like 64 to 32, 128 or other desired values but the last layer must have nodes equal to the number of classes you are using,as in here it is 3.**

In [ ]:
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#model.add(Dense(32))
#model.add(Activation('relu'))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

**This cell below performs the training. The callback function is used to stop the training when no improvement occurs.**

In [ ]:
%%time
h = model.fit(
    train_batches,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=test_batches,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[
        #tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4),
        tf.keras.callbacks.ModelCheckpoint(filepath = '/kaggle/working/Model_{val_accuracy:.3f}.h5', save_best_only=True,
                                          save_weights_only=False, monitor='val_accuracy')
    ])

**The true label for test images are loaded here and processed necessarily.**

In [ ]:
test_batches = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=144,
    classes=['BENIGN', 'MALIGNANT', 'NORMAL'])
    #color_mode="grayscale")

In [ ]:
test_imgs, test_labels = next(test_batches)
rounded_labels = np.argmax(test_labels, axis=-1)

**The best performing model is loaded here.**

In [ ]:
test_model = load_model('./Model_0.797.h5')

**Here the prediction is performed.**

In [ ]:
predictions = test_model.predict(test_batches, steps=1, verbose=0)

In [ ]:
predictions

In [ ]:
rounded_prediction = np.argmax(predictions, axis=-1)

In [ ]:
for i in rounded_prediction:
    print(i)

**The confusion matrix shows the result of the model**

In [ ]:
cm = confusion_matrix(y_true=rounded_labels, y_pred=rounded_prediction)

In [ ]:
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
cm_plot_labels = ['BENIGN', 'MALIGNANT', 'NORMAL']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='confusion_matrix')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true=rounded_labels, y_pred=rounded_prediction))

**This is the accuracy curve**

In [ ]:
accs = h.history['accuracy']
val_accs = h.history['val_accuracy']

plt.plot(range(len(accs)),accs, label = 'Training_accuracy')
plt.plot(range(len(accs)),val_accs, label = 'Testing_accuracy')
plt.xlabel('x-axis (no.of epoch)')
plt.ylabel('y-axis (accuracy_value)')
plt.legend()
plt.show()

**This is the loss curve.These accuracy and loss curves only generate when the training is performed. If just load a pre-trained best performing model without training the model then running thses cells will generate error.**

In [ ]:
accs = h.history['loss']
val_accs = h.history['val_loss']

plt.plot(range(len(accs)),accs, label = 'Training_loss')
plt.plot(range(len(accs)),val_accs, label = 'Testing_loss')
plt.xlabel('x-axis (no.of epoch)')
plt.ylabel('y-axis (loss_value)')
plt.legend()
plt.show()

**Here necessary functions are defined to load a single image to perform the prediction**

In [ ]:
from PIL import Image
import numpy as np
from skimage import transform
def load(filename):
    img = Image.open(filename)
    img = np.array(img).astype('float32')/255
    img = transform.resize(img, (64, 64, 3))
    img = np.expand_dims(img, axis=0)
    return img

**Here the image is loaded and prediction is performed. Enter the path of the desired image below inside the ('  ')**

In [ ]:
image = load('../input/modiified/Modified_Dataset/Test/MALIGNANT/10.jpg')
prediction = test_model.predict(image)
prediction

**This cell block will give the predicted output**

In [ ]:
rounded_prediction = np.argmax(prediction, axis=-1)
if rounded_prediction == 0:
    print("Benign")
elif rounded_prediction == 1:
    print("Malignant")
elif rounded_prediction == 2:
    print("Normal")